In [1]:
# All the necessary libraries

import echonestkeys as secret
import json
import urllib2
import time 

from bs4 import BeautifulSoup
import requests
import html5lib

import pandas as pd
import numpy as np

from fuzzywuzzy import fuzz, process

In [2]:
# Somewhere to start and get a good number of artists
def get_soup(url):
    response = requests.get(url) 
    soup = None
    if response.status_code == 200:
        page = response.text
        soup = BeautifulSoup(page, 'html5lib')
    else:
        print 'Unresponsive. Trying Again...'
        time.sleep(0.5)
        return get_soup(url)
    return soup


In [3]:

def make_albumdf(year):
    soup = get_soup('https://en.wikipedia.org/wiki/List_of_%d_albums' % (year))
    tables = soup.find_all('table', attrs={'wikitable'})
    df = pd.DataFrame()
    for table in tables:
        date = ' '.join(table.find('td').text.splitlines())
        header = False
        for tr in table('tr'):
            
            first = tr.find('td')
            text = first.text.splitlines() if (first is not None) else 'heading'
            if text is not 'heading':
                row = []
                if len(text) == 2:
                    date = ' '.join(text)
                    row.append(date)
                    for cell in tr.find_all('td'):
                        if ' '.join(cell.text.splitlines())!=date:
                            row.append(cell.text)
                else:
                    row.append(date)
                    for cell in tr.find_all('td'):
                        row.append(cell.text)
                
                if len(row)==7:
                    row.pop()
                    df = df.append(pd.Series(row),ignore_index=True)
    return df


In [4]:
def get_json(url):
    response = requests.get(url)
    if response.status_code==200:
        return json.loads(response.text)
    else:
        print 'Rate Limited url: %s' % url
        return None

def fuzzy_match(a, names):
    if a == names[0]:
        return 0
    if a.lower() == names[0].lower():
        return 0
    best = max([(fuzz.ratio(a, n),n) for n in names])
    if best[0]>0.7:
        return names.index(best[1])
    else:
        return -1
        

In [5]:
# Some artists to work with
artists=sorted(set(make_albumdf(2013)[1]))

In [14]:
# Spotify search for one artist

def get_spotify_artist(artist):
    q = '+'.join(artist.split()).strip()
    url = "https://api.spotify.com/v1/search?q=%s&type=artist" % q
    results = get_json(url)['artists']['items']
    if results != []:
        names = [r['name'] for  r in results]
        index = fuzzy_match(artist, names)
        if index != -1:
            return results[index]['id'] #id
    else:
        return None
    
        
echo_limit = 0
           
def get_echo_artist(artist):
    q = '+'.join(artist.split()).strip()
    url = 'http://developer.echonest.com/api/v4/artist/search?api_key=%s&\
format=json&name=%s' % (secret.apikey(), q)
    
    global echo_limit
    echo_limit += 1
    if echo_limit%119 == 0:
        time.sleep(60)

    response = get_json(url)['response']
    results = response['artists']
    if results != []:
        names = [r['name'] for r in results]
        index = fuzzy_match(artist, names)
        if index != -1:
            return results[index]['id']
    else:
        return None


In [7]:
def get_albums(artist_id):
    url = 'https://api.spotify.com/v1/artists/%s/albums?album_type=album&limit=50' % artist_id
    albums = get_json(url)
    if albums['items']==[]:
        print 'No albums for %s' % artist_id
        return None
    
    album_names = [album['name'] for album in albums['items']]
    album_ids_dup = np.array([album['id'] for album in albums['items']])

    inds = [album_names.index(a) for a in set(album_names)]
    album_ids = album_ids_dup[inds]
    return album_ids

In [20]:
def get_tracks(album_id):
    url = 'https://api.spotify.com/v1/albums/%s/tracks?limit=50' % album_id
    tracks = get_json(url)
    return [t['uri'] for t in tracks['items']]


In [132]:
get_tracks(get_albums(get_artist(artists[0]))[0])[0]

u'spotify:track:1FeOrTzQulW3SqdIlGfjKe'

In [10]:
# EchoNest search for same artist
for artist in artists:
#     try:
    sp = get_spotify_artist(artist)
    ec = get_echo_artist(artist)
    if ec != None:
        if sp != None:
            if sp.lower() != ec.lower():
                print 'spotify: ',sp
                print 'echo: ',ec
                print fuzz.ratio(sp, ec)
                print fuzz.partial_ratio(sp,ec)
                print fuzz.WRatio(sp, ec)
                print '\n'
        else:
            print 'artist :',artist
            print 'spotify: None'
    else:
        print 'artist :',artist
        print 'echo: none'
    print '\n'
#     except:
#         print artist
#         print '\n'





spotify:  98
echo:  98 Degrees
33
100
90












spotify:  A$AP Ferg
echo:  ASAP Ferg
89
88
89




















































































spotify:  Babasonicos
echo:  Babasónicos
91
90
95






















artist : Ben Harper and Charlie Musselwhite
spotify: None




































































spotify:  Brown Eyed Girls
echo:  Brown Eyed Girls&SeeYa
84
100
95


































Rate Limited url: http://developer.echonest.com/api/v4/artist/search?api_key=0GGBLPK4OHMQIJYQ2&format=json&name=Chase+&+Status


TypeError: 'NoneType' object has no attribute '__getitem__'

In [15]:
get_echo_artist('2 chainz')

u'ARPVJVZ12FE085D726'

In [18]:
get_albums(get_spotify_artist('Ariana Grande'))

array([u'2ZnzBwKw4e2SHpGvOTWnj4', u'2LFT0OnfZzsvjcU7bNMM7W',
       u'6WQzwx2a6Wq6rDoEU3TaNE', u'6czdbbMtGbAkZ6ud2OMTcg',
       u'6dYDqMHA4COCFC0TfCiUCj'], 
      dtype='<U22')

In [21]:
get_tracks('6czdbbMtGbAkZ6ud2OMTcg')

[u'spotify:track:5SmXEPnevlRjBPWBG7oKIi',
 u'spotify:track:6NmCCgnQcxKHPOAKgd9Nth',
 u'spotify:track:47rzxY9wSpomHE1gT94jIT',
 u'spotify:track:7JmIjOsRish3vYBSLTytEC',
 u'spotify:track:0CN72fWya38TPEQzrsr27w',
 u'spotify:track:1FSfCzJd8aSQ0nqTSbN4PO',
 u'spotify:track:2CcQDI2IDhHbmBIFup58JE',
 u'spotify:track:0S4RKPbRDA72tvKwVdXQqe',
 u'spotify:track:6d8XhbAw4IoKPnVD07Sb9b',
 u'spotify:track:442j8VxaB60dWf9cBFuX5w',
 u'spotify:track:6AviHKu3ydzAePBmzEi62v',
 u'spotify:track:0vyQdRbRJRRY8GGuM8eVVI']

In [ ]:
       
echo_limit = 0
def rate_limiter():
    global echo_limit
    echo_limit += 1
    if echo_limit%119 == 0:
        time.sleep(60)

           
def get_echo_artist(artist):
    q = '+'.join(artist.split()).strip()
    url = 'http://developer.echonest.com/api/v4/artist/search?api_key=%s&\
format=json&name=%s' % (secret.apikey(), q)
    rate_limiter()
    response = get_json(url)['response']
 
    try:
        results = response['artists']
    except:
        print response['status']['message']
        pass
        
    if results != []:
        names = [r['name'] for r in results]
        index = fuzzy_match(artist, names)
        if index != -1:
            return results[index]['name']
    else:
        return None

        

In [22]:
# [u'genres',
#  u'name',
#  u'external_urls',
#  u'popularity',
#  u'uri',
#  u'href',
#  u'followers',
#  u'images',
#  u'type',
#  u'id']



# [u'name',
#  u'external_urls',
#  u'uri',
#  u'explicit',
#  u'preview_url',
#  u'track_number',
#  u'disc_number',
#  u'href',
#  u'artists',
#  u'duration_ms',
#  u'type',
#  u'id',
#  u'available_markets']